# Machine Learning Project 2020-2021

# Setup Environment
Before running this notebook, the user needs to configure a separate conda environment in order to reproduce the results obtained. An environment.yml file is provided in the project repository with this objective. By creating a conda environment through the yml file, the user will be able to run the whole project without issues. Besides, the user also need to install an additional package from a personal github repository since it is not available in pypi nor in conda repositories.

In order to create the environment the user needs to run the following commands in the anaconda prompt: **conda env create -f \<path to environment.yml>**. Afterwards, the user needs to activate the created environment: **conda activate ml** and initialize the jupyter notebook: **jupyter notebook**.

In [7]:
import sys
#!{sys.executable} -m pip install git+https://github.com/johnruiz24/ML_Project 

## 1 - Importing Packages

In [1]:
import os
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from datetime import date, timedelta
import matplotlib.pyplot as plt
#EDA Libraries
import dtale
import sweetviz as sv
import pandas_profiling
from pandas_profiling import ProfileReport
#ML Libraries
from sklearn.linear_model import LassoCV
from sklearn.linear_model import RidgeClassifierCV
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import (Normalizer,
                                   MinMaxScaler,
                                   LabelEncoder,
                                   RobustScaler,
                                   StandardScaler,OneHotEncoder,OrdinalEncoder)
from sklearn.linear_model import RidgeClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error, median_absolute_error

import sklearn.preprocessing as prep
import sklearn.covariance as cov #Elliptic Envelop
from scipy.stats import chi2

%matplotlib inline
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format = 'retina'

# 2 - Data Import and integration

In [2]:
train =pd.read_excel(os.path.join(os.getcwd(),'Train.xlsx'))

In [10]:
#train

In [3]:
test=pd.read_excel(os.path.join(os.getcwd(),'Test.xlsx'))

# 3 - Exploring Data

In [3]:
train.head()

,CITIZEN_ID,Name,Birthday,Native Continent,Marital Status,Lives with,Base Area,Education Level,Years of Education,Employment Sector,Role,Working Hours per week,Money Received,Ticket Price,Income
0,12486,Mr. Adam Glover,"July 1,2003",Europe,Married,Wife,Northbury,High School + PostGraduation,13,Private Sector - Services,Repair & constructions,40,0,2273,1
1,12487,Mr. Cameron McDonald,"January 25,2006",Europe,Married,Wife,Northbury,Professional School,12,Public Sector - Others,Repair & constructions,40,0,0,1
2,12488,Mr. Keith Davidson,"May 10,2009",Europe,Married,Wife,Northbury,Professional School,12,Private Sector - Services,Sales,46,0,2321,1
3,12489,Mr. Alexander Gill,"March 25,1985",Europe,Married,Wife,Northbury,High School - 2nd Cycle,11,Private Sector - Services,Security,37,5395,0,1
4,12490,Mr. Neil Piper,"May 29,2015",Europe,Single,Other Family,Northbury,PhD,21,Self-Employed (Individual),Professor,45,0,0,1


In [146]:
train.describe()

,CITIZEN_ID,Years of Education,Working Hours per week,Money Received,Ticket Price,Income
count,22400.000000,22400.000000,22400.000000,22400.000000,22400.000000,22400.000000
mean,23685.500000,13.173884,40.483795,1324.915357,109.145313,0.237098
std,6466.467351,2.512451,12.370921,9227.771813,500.208904,0.425313
min,12486.000000,2.000000,1.000000,0.000000,0.000000,0.000000
25%,18085.750000,12.000000,40.000000,0.000000,0.000000,0.000000
50%,23685.500000,13.000000,40.000000,0.000000,0.000000,0.000000
75%,29285.250000,15.000000,45.000000,0.000000,0.000000,0.000000
max,34885.000000,21.000000,99.000000,122999.000000,5358.000000,1.000000


In [13]:
#Dimension of the training dataset
train.shape

(22400, 15)

In [14]:
#Dimension of the testing data
test.shape

(10100, 14)

In [15]:
#Data types for the train dataset 
train.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22400 entries, 0 to 22399
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CITIZEN_ID              22400 non-null  int64 
 1   Name                    22400 non-null  object
 2   Birthday                22400 non-null  object
 3   Native Continent        22400 non-null  object
 4   Marital Status          22400 non-null  object
 5   Lives with              22400 non-null  object
 6   Base Area               22400 non-null  object
 7   Education Level         22400 non-null  object
 8   Years of Education      22400 non-null  int64 
 9   Employment Sector       22400 non-null  object
 10  Role                    22400 non-null  object
 11  Working Hours per week  22400 non-null  int64 
 12  Money Received          22400 non-null  int64 
 13  Ticket Price            22400 non-null  int64 
 14  Income                  22400 non-null  int64 
dtypes:

In [16]:
#Data types for the testing dataset 
test.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10100 entries, 0 to 10099
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   CITIZEN_ID              10100 non-null  int64 
 1   Name                    10100 non-null  object
 2   Birthday                10100 non-null  object
 3   Native Continent        10100 non-null  object
 4   Marital Status          10100 non-null  object
 5   Lives with              10100 non-null  object
 6   Base Area               10100 non-null  object
 7   Education Level         10100 non-null  object
 8   Years of Education      10100 non-null  int64 
 9   Employment Sector       10100 non-null  object
 10  Role                    10100 non-null  object
 11  Working Hours per week  10100 non-null  int64 
 12  Money Received          10100 non-null  int64 
 13  Ticket Price            10100 non-null  int64 
dtypes: int64(5), object(9)
memory usage: 1.1+ MB


In [17]:
#Check duplicates
any(train.duplicated())

False

In [18]:
#Check duplicates
any(test.duplicated())

False

In [6]:
# after a first inspection of the train data excel file, we found out that some columns are the following string: '?' so bellow we check the quantity of '?' string and what columns have them. 
def checkdf_character(df,char): 
    for i in range(len(df.columns)):
        #Printing the column names, number of rows & percentage of rows from the dataset
        #Further details will be printed in the next lines through the utilization of EDA tools
        size=len(df.loc[df.iloc[:,i].astype(str).str.startswith('?')])
        if size>0:
            print('------------------------------------------')
            print('Column: {}{}Number of Rows: {}{}Percentage: {}%'.format(df.columns[i],'\n',size,'\n',round(size/len(df)*100,2)))

In [20]:
#Data inspection shows that the train dataset has rows with question mark '?' character
checkdf_character(train,'?')

------------------------------------------
Column: Base Area
Number of Rows: 395
Percentage: 1.76%
------------------------------------------
Column: Employment Sector
Number of Rows: 1264
Percentage: 5.64%
------------------------------------------
Column: Role
Number of Rows: 1271
Percentage: 5.67%


In [21]:
#Data inspection shows that the test dataset has rows with question mark '?' character
checkdf_character(test,'?')

------------------------------------------
Column: Base Area
Number of Rows: 187
Percentage: 1.85%
------------------------------------------
Column: Employment Sector
Number of Rows: 570
Percentage: 5.64%
------------------------------------------
Column: Role
Number of Rows: 570
Percentage: 5.64%


In [22]:
#Frequencies of number of missing values across rows
train.isna().apply(lambda x:sum(x), axis=1).value_counts().sort_index()

0    22400
dtype: int64

In [23]:
#Frequencies of number of missing values across rows
test.isna().apply(lambda x:sum(x), axis=1).value_counts().sort_index()

0    10100
dtype: int64

- Identify the outliers before going to the data preparation ? 

In [24]:
#dtale.show(train)

# 4 - Modify the Data / Data Preparation 

# 4.1 Creating two new features from existing variables.

### 4.1.1 - Creating Gender from the variable Name, 0 for male and 1 for female.

In [4]:
train_upd = train.copy()

In [7]:
checkdf_character(train_upd,'?')

------------------------------------------
Column: Base Area
Number of Rows: 395
Percentage: 1.76%
------------------------------------------
Column: Employment Sector
Number of Rows: 1264
Percentage: 5.64%
------------------------------------------
Column: Role
Number of Rows: 1271
Percentage: 5.67%


In [8]:
train_upd.insert(2,'Gender',train_upd['Name'].str.slice(0,4).str.replace('.','').str.replace(' ',''))

In [9]:
train_upd['Gender']=train_upd['Gender'].apply(lambda i: 0 if i=='Mr' else 1)

In [10]:
train_upd.drop('Name',axis=1,inplace=True)

In [11]:
train_upd['Gender'].unique()

array([0, 1], dtype=int64)

### 4.1.2 - Creating Age from the variable Birthday

In [12]:
train_upd.insert(4,'Age',pd.Series([]))
for i,j in enumerate(list(train_upd['Birthday'].values)):
    try:
        train_upd['Age'][i]=np.round((datetime.now()+timedelta(days=10228)-datetime.strptime(j.replace(' ',''),'%B%d,%Y'))/timedelta(days=365.2425))
    except ValueError:
        train_upd['Age'][i]=np.round((datetime.now()+timedelta(days=10228)-datetime.strptime(j.replace(' ','').replace('9','8'),'%B%d,%Y'))/timedelta(days=365.2425))

In [13]:
train_upd['Age']=train_upd['Age'].astype('uint8')

In [14]:
train_upd.drop('Birthday',axis=1,inplace=True)

In [15]:
train_upd

,CITIZEN_ID,Gender,Native Continent,Age,Marital Status,Lives with,Base Area,Education Level,Years of Education,Employment Sector,Role,Working Hours per week,Money Received,Ticket Price,Income
0,12486,0,Europe,45,Married,Wife,Northbury,High School + PostGraduation,13,Private Sector - Services,Repair & constructions,40,0,2273,1
1,12487,0,Europe,43,Married,Wife,Northbury,Professional School,12,Public Sector - Others,Repair & constructions,40,0,0,1
2,12488,0,Europe,40,Married,Wife,Northbury,Professional School,12,Private Sector - Services,Sales,46,0,2321,1
3,12489,0,Europe,64,Married,Wife,Northbury,High School - 2nd Cycle,11,Private Sector - Services,Security,37,5395,0,1
4,12490,0,Europe,34,Single,Other Family,Northbury,PhD,21,Self-Employed (Individual),Professor,45,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22395,34881,1,Europe,26,Divorced,Children,Northbury,Professional School,12,Private Sector - Services,Cleaners & Handlers,40,0,0,0
22396,34882,0,Europe,24,Single,Children,Northbury,Professional School,12,Private Sector - Services,Other services,35,0,0,0
22397,34883,1,Europe,26,Single,Other Family,Northbury,Masters,17,Public Sector - Others,Professor,40,2674,0,0
22398,34884,0,Europe,37,Single,Other Family,Northbury,Bachelors + PostGraduation,16,Private Sector - Services,Repair & constructions,90,0,0,1


# 4.2 - Filling the values with '?'

In [16]:
# since we are using IterativeImputer() to fill the '?' string values we need to change them to NaN's so the algorigthm works.
for i in [8,11,12]:
    aIndex=np.array(train_upd[train_upd.iloc[:,i].astype(str).str.startswith('?')].iloc[:,i].index)
    for j in aIndex:
        train_upd.iloc[j,i]=np.nan

train_upd.isna().sum()

CITIZEN_ID                0
Gender                    0
Native Continent          0
Age                       0
Marital Status            0
Lives with                0
Base Area                 0
Education Level           0
Years of Education        0
Employment Sector         0
Role                      0
Working Hours per week    0
Money Received            0
Ticket Price              0
Income                    0
dtype: int64

In [19]:
#Creating a data frame with the categorical variables.
#cat_col = train_upd.select_dtypes(include=['object']).columns.values # Because it does encode numeric features like years of education. 
cat_col = train_upd[['Native Continent','Marital Status','Lives with','Base Area','Education Level','Employment Sector','Role']].columns.values
cat_col

array(['Native Continent', 'Marital Status', 'Lives with', 'Base Area',
       'Education Level', 'Employment Sector', 'Role'], dtype=object)

In [20]:
encoder = OrdinalEncoder()

In [21]:
mice = IterativeImputer()

In [22]:
def encode(data):
    reshape = np.array(data.dropna()).reshape(-1,1)
    ordinal = encoder.fit_transform(reshape)
    data.loc[data.notnull()] = np.squeeze(ordinal)
    return data

In [23]:
# Not running for now.
for col in cat_col: 
    encode(train_upd[col])

In [24]:
#Filling NaN according to Mice algorigthm. 
train_upd=pd.DataFrame(np.round(mice.fit_transform(train_upd)),columns=train_upd.columns)

In [25]:
# double check to what was done.
train_upd.isna().sum()

CITIZEN_ID                0
Gender                    0
Native Continent          0
Age                       0
Marital Status            0
Lives with                0
Base Area                 0
Education Level           0
Years of Education        0
Employment Sector         0
Role                      0
Working Hours per week    0
Money Received            0
Ticket Price              0
Income                    0
dtype: int64

In [41]:
#train_upd

# 4.3 Removing the Outliers - Using mahalanobis

In [42]:
#John needs to check. 
main_data = train_upd.copy().reset_index()
main_data['id'] = main_data.index

In [43]:
metric_data = main_data.loc[:,(np.array(main_data.dtypes=="float64")) | (np.array(main_data.dtypes=="int64"))]

data = metric_data.set_index("id")

yj = prep.PowerTransformer(method='yeo-johnson')
qt = prep.QuantileTransformer(n_quantiles=500, output_distribution='normal', random_state=np.random.RandomState(104))

X_trans_yj = yj.fit_transform(data)
X_trans_qt = qt.fit_transform(data)

# Merging
X_trans = np.concatenate((X_trans_yj, X_trans_qt), axis=1)

# Creating DataFrame
merge_labels = list(map(lambda x: "yj_" + x, data.columns.to_list()))+list(map(lambda x: "qt_" + x, data.columns.to_list()))
trans_df = pd.DataFrame(X_trans, columns=merge_labels, index=data.index)

In [44]:
transqt_df = trans_df.loc[:,[i for i in trans_df.columns.to_list() if "qt" in i]]
transyj_df = trans_df.loc[:,[i for i in trans_df.columns.to_list() if "yj" in i]]

In [45]:
def out_mahalanobis(data, random, contamination=0.03):
    n = data.shape[0]
    df = data.shape[1]
    # Obtain mahalanobis distance of each observation to the center and then sort it
    ee_out = cov.EllipticEnvelope(contamination=0.2, support_fraction=0.95, random_state=np.random.RandomState(304)).fit(data)
    mahalan = ee_out.mahalanobis(data)
    # Obtain chi-square percentiles of each observations
    quantiles = chi2.ppf(q=np.fromfunction(lambda x:(x-0.5)/n, (n,)), df=df)
    # Obtain labels
    labels = ee_out.predict(data)    
    return mahalan, quantiles, labels

In [46]:
mqt, qqt, lqt = out_mahalanobis(transqt_df, 0)

In [47]:
myj, qyj, lyj = out_mahalanobis(transyj_df, 0)

In [48]:
lqt_lyj = pd.DataFrame({'lqt': lqt, 'lyj': lyj})

In [49]:
# checking outliers precentage
inl = lqt_lyj.loc[(lqt_lyj['lqt'] == 1) & (lqt_lyj['lyj'] == 1)].sum()['lqt']
'Outliers percentage: {:4.2f}%'.format(100 - inl/len(lqt_lyj) * 100)

'Outliers percentage: 27.08%'

In [50]:
# creating mask for outliers' drop
lqt_lyj.loc[(lqt_lyj['lqt'] == 1) & (lqt_lyj['lyj'] == 1), 'mask'] = True
lqt_lyj.fillna(False, inplace=True)
lqt_lyj.drop(columns=['lqt', 'lyj'], inplace=True)

# - Trying with 1 Dataset 

In [30]:
onehot_test = train_upd.copy()

In [33]:
onehot_data = onehot_test.iloc[:,:-1]
onehot_target = onehot_test.iloc[:,-1]

In [34]:
onehot_data.head(1)

,CITIZEN_ID,Gender,Native Continent,Age,Marital Status,Lives with,Base Area,Education Level,Years of Education,Employment Sector,Role,Working Hours per week,Money Received,Ticket Price
0,12486,0,Europe,45,Married,Wife,Northbury,High School + PostGraduation,13,Private Sector - Services,Repair & constructions,40,0,2273


In [37]:
#scaler = MinMaxScaler() #RobustScaler()
#scaled_feat = scaler.fit_transform(onehot_data[metric_features])
#scaled_feat

In [38]:
#onehot_data[metric_features] = scaled_feat
#onehot_data.head(1)

# 4.4 - Divinding the data in 3 different data sets - because we have 3 distinct groups, Volunteers, Essencial and ticket buyers. 

In [26]:
train_upd = train_upd.astype(int)

In [27]:
train_upd.head()

,CITIZEN_ID,Gender,Native Continent,Age,Marital Status,Lives with,Base Area,Education Level,Years of Education,Employment Sector,Role,Working Hours per week,Money Received,Ticket Price,Income
0,12486,0,3,45,1,5,29,2,13,3,11,40,0,2273,1
1,12487,0,3,43,1,5,29,14,12,5,11,40,0,0,1
2,12488,0,3,40,1,5,29,14,12,3,12,46,0,2321,1
3,12489,0,3,64,1,5,29,4,11,3,13,37,5395,0,1
4,12490,0,3,34,5,3,29,11,21,7,10,45,0,0,1


In [28]:
train_upd_a=train_upd[(train_upd['Ticket Price']==0) & (train_upd['Money Received']==0)].copy() #Volunteers
train_upd_b=train_upd[train_upd['Money Received']>0].copy()#received money
train_upd_c=train_upd[train_upd['Ticket Price']>0].copy()#buying entry

In [53]:
# DO NOT RUN, It's here fow now, just for reference.
#group a
#df1=train_upd_a.copy()
#for c in cat_col:
    #encode(df1[c])

In [54]:
## DO NOT RUN, It's here fow now, just for reference.
#group b
#df2=train_upd_b.copy()
#for c in cat_col:
    #encode(df2[c])

In [55]:
# DO NOT RUN, It's here fow now, just for reference.
#group c
#df3=train_upd_c.copy()
#for c in cat_col:
    #encode(df3[c])

### Scaling

In [ ]:
#robust_scaler = RobustScaler()

In [29]:
min_max = train_upd_a

In [30]:
# Use MinMaxScaler to scale the data
scaler = MinMaxScaler()
scaled_feat = scaler.fit_transform(min_max)
scaled_feat

array([[0.00000000e+00, 0.00000000e+00, 7.50000000e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.33940530e-04, 0.00000000e+00, 7.50000000e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.78587374e-04, 0.00000000e+00, 7.50000000e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.99866059e-01, 0.00000000e+00, 7.50000000e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.99955353e-01, 0.00000000e+00, 7.50000000e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00, 5.00000000e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [35]:
min_max[metric_features] = scaled_feat
min_max.head()

NameError: name 'metric_features' is not defined

# We could also try lasso regression, it's more accurate and easier to understand. 

# 4.5 - RFE - Recursive Feature Selection

In [56]:
# We should choose between one of the methods, or RFE or Ridge or Lasso.

In [57]:
def rfe_group(df):
    data = df.iloc[:,:-1]
    target = df.iloc[:,-1]
    nof_list=np.arange(1,len(data.columns)+1)  
    high_score=0
    nof=0           
    score_list =[]
    #dividing the data sets into train and validation. 
    for n in range(len(nof_list)):
        X_train, X_test, y_train, y_test = train_test_split(data,target, test_size = 0.2, random_state = 0,stratify = target)
        
        #Selecting the model for the Feature selection.
        model = LogisticRegression()
        rfe = RFE(model,nof_list[n])
        X_train_rfe = rfe.fit_transform(X_train,y_train)
        X_test_rfe = rfe.transform(X_test)
        model.fit(X_train_rfe,y_train)

        score = model.score(X_test_rfe,y_test)
        score_list.append(score)

        if(score>high_score):
            high_score = score
            nof = nof_list[n]
    print("Optimum number of features: %d" %nof)
    print('\n')
    print("Score with %d features: %f" % (nof, high_score))
    rfe = RFE(estimator = model, n_features_to_select = nof)
    X_rfe = rfe.fit_transform(X = data, y = target)
    model = LogisticRegression().fit(X = X_rfe,y = target)
    selected_features = pd.Series(rfe.support_, index = data.columns)
    print(selected_features)
    ridge = RidgeClassifierCV().fit(X = data,y = target)
    coef_ridge = pd.Series(ridge.coef_[0], index = data.columns)
    return coef_ridge

In [58]:
#Ploting the features importance with ridge regression. 
def plot_importance(coef,name):
    imp_coef = coef.sort_values()
    plt.figure(figsize=(8,10))
    imp_coef.plot(kind = "barh")
    plt.title("Feature importance using " + name)
    plt.show()

In [123]:
coef_ridge_1 = rfe_group(train_upd_a)
#plot_importance(coef_ridge_1,'Group A -> Volunteers - No Payment')

Optimum number of features: 8


Score with 8 features: 0.839867
CITIZEN_ID                False
Name                      False
Gender                    False
Birthday                  False
Age                        True
Native Continent           True
Marital Status             True
Lives with                 True
Base Area                  True
Education Level           False
Years of Education         True
Employment Sector          True
Role                       True
Working Hours per week    False
Money Received            False
Ticket Price              False
dtype: bool


In [124]:
# Selecting the Features for the Volunteers dataset.
train_upd_a.drop(['Gender','Birthday','Ticket Price','Money Received','Name','Education Level','Working Hours per week'],axis=1,inplace=True)

In [125]:
train_upd_a.set_index('CITIZEN_ID',inplace=True) # Check later and after making changes if ID is still correct. 

In [126]:
train_upd_a

,Age,Native Continent,Marital Status,Lives with,Base Area,Years of Education,Employment Sector,Role,Income
CITIZEN_ID,,,,,,,,,
12487,43,3,1,5,28,12,4,10,1
12490,34,3,5,3,28,21,6,9,1
12491,24,3,5,1,28,15,2,3,0
12493,37,3,2,4,4,11,2,8,0
12494,24,3,5,1,28,13,1,6,0
...,...,...,...,...,...,...,...,...,...
34880,36,3,1,5,28,16,2,11,1
34881,26,3,0,1,28,12,2,3,0
34882,24,3,5,1,28,12,2,8,0


In [141]:
coef_ridge_2 = rfe_group(train_upd_b)
#plot_importance(coef_ridge_2,'Group B -> Were invited (Money Received)')

Optimum number of features: 12


Score with 12 features: 0.858311
CITIZEN_ID                False
Name                      False
Gender                     True
Birthday                  False
Age                        True
Native Continent           True
Marital Status             True
Lives with                 True
Base Area                  True
Education Level            True
Years of Education         True
Employment Sector          True
Role                       True
Working Hours per week     True
Money Received             True
Ticket Price              False
dtype: bool


In [64]:
# Selecting the Features for the essencials dataset.
train_upd_b.drop(['Birthday','Ticket Price','Money Received','Name','Base Area','Age'],axis=1,inplace=True) # We are letting Education lvl stay since it's has some weight. 

In [65]:
train_upd_b.set_index('CITIZEN_ID',inplace=True) # Check later and after making changes if ID is still correct. 

In [66]:
# Checking score after removing the serveral variables. 
rfe_group(train_upd_b)

Optimum number of features: 2


Score with 2 features: 0.735695
Gender                    False
Native Continent          False
Marital Status            False
Lives with                 True
Education Level           False
Years of Education         True
Employment Sector         False
Role                      False
Working Hours per week    False
dtype: bool


Gender                    0.037615
Native Continent         -0.019923
Marital Status           -0.087348
Lives with                0.117337
Education Level          -0.007872
Years of Education        0.116507
Employment Sector         0.006597
Role                      0.008874
Working Hours per week    0.008842
dtype: float64

In [140]:
coef_ridge_3 = rfe_group(train_upd_c)
#plot_importance(coef_ridge_3,'Group C -> Money offering - Ticket Price')

Optimum number of features: 3


Score with 3 features: 0.764151
CITIZEN_ID                False
Name                      False
Gender                    False
Birthday                  False
Age                       False
Native Continent          False
Marital Status             True
Lives with                 True
Base Area                 False
Education Level           False
Years of Education         True
Employment Sector         False
Role                      False
Working Hours per week    False
Money Received            False
Ticket Price              False
dtype: bool


In [68]:
# Selecting the Features for the essencials dataset.
train_upd_c.drop(['Birthday','Ticket Price','Money Received','Name','Base Area','Age','Role','Education Level'],axis=1,inplace=True) # We are letting Education lvl stay since it's has some weight. 

In [69]:
train_upd_c.set_index('CITIZEN_ID',inplace=True) # Check later and after making changes if ID is still correct. 

In [70]:
# Checking score after removing the serveral variables. 
rfe_group(train_upd_c)

Optimum number of features: 3


Score with 3 features: 0.764151
Gender                    False
Native Continent          False
Marital Status             True
Lives with                 True
Years of Education         True
Employment Sector         False
Working Hours per week    False
dtype: bool


Gender                   -0.007362
Native Continent          0.001455
Marital Status           -0.100274
Lives with                0.166116
Years of Education        0.108189
Employment Sector         0.043221
Working Hours per week    0.007302
dtype: float64

# 5 - Modelling - Create a predictive model

## 5.1 Running logistic regression

### 5.1.1 Dataset Volunteers

In [127]:
#Spliting train and validation data
 # Volunteers dataset
data_1 = train_upd_a.iloc[:,:-1]
target_a = train_upd_a.iloc[:,-1]

In [128]:
# dividing Volunteers dataset into train and validation. 
X_train, X_val, y_train, y_val = train_test_split(data_1,target_a, test_size=0.2, random_state=0, stratify=target_a)  

In [129]:
log_model = LogisticRegression()

In [130]:
log_model.fit(X_train,y_train)

LogisticRegression()

In [131]:
y_pred = log_model.predict(X_val)
#y_pred

In [132]:
pred_prob = log_model.predict_proba(X_val)
#pred_prob

In [133]:
log_model.coef_

array([[ 0.02116777,  0.09420172, -0.30846698,  0.43969419,  0.00391424,
         0.3404667 , -0.01976371,  0.02662208]])

In [134]:
cm = confusion_matrix(y_val, y_pred)
cm

array([[3018,  155],
       [ 467,  263]], dtype=int64)

The confusion matrix in sklearn is presented in the following format: <br>
[ [ TN  FP  ] <br>
    [ FN  TP ] ]

In [135]:
accuracy = accuracy_score(y_val, y_pred)
accuracy

0.8406354086600051

In [136]:
precision = precision_score(y_val, y_pred)
precision

0.6291866028708134

In [137]:
recall_score(y_val, y_pred)

0.36027397260273974

In [138]:
f1 = f1_score(y_val, y_pred)
f1

0.45818815331010454

In [139]:
print('Results on the validation set:')
print(
    
    classification_report(y_true = y_val, y_pred = y_pred)
)

Results on the validation set:
              precision    recall  f1-score   support

           0       0.87      0.95      0.91      3173
           1       0.63      0.36      0.46       730

    accuracy                           0.84      3903
   macro avg       0.75      0.66      0.68      3903
weighted avg       0.82      0.84      0.82      3903



### Performance measures

In [84]:
# Not sure R2 and Adjusted R2 are the best algorithms to check logistic regression.

<a class="anchor" id="rsquare">
    
###  $R^{2}$ Score

</a>

__When to use?__ <br>
When we want to measure the amount of variance in the target variable that can be explained by our model. <br>
It gives the degree of variability in the target variable that is explained by the model or the independent variables. <br>
If this value is 0.7, then it means that the independent variables explain 70% of the variation in the target variable. <br>

__Definition:__ <br>
R^2 (coefficient of determination) regression score function.

__Interpretation:__ <br>
Best possible score is 1.0 and it can be negative (because the model can be arbitrarily worse). 

__Parameters:__ <br>
_y_true_: Ground truth (correct) target values; <br>
_y_pred_: Estimated target values; <br>


In [85]:
r2_score(y_val, y_pred)
#when we want to measure variance in target value.
#when you apply R2 score to different number of variables the R2 score changes,
# not good to compare. We can use Adjusted R score.

-0.05145383350098642

<a class="anchor" id="adjusted">
    
### 1.2. Adjusted $R^{2}$ Score

</a>

There is no direct way to obtain the adjusted R^2 using sklearn, but we can apply the formula:
<img src="adj_r2.png" alt="Drawing" style="width: 300px;"/> <br>


where n stands for the sample size and p for the number of the regressors.

In [86]:
r2 = r2_score(y_val, y_pred)
n = len(y_val)
p = len(X_train.columns)

def adj_r2 (r2,n,p):
    return 1-(1-r2)*(n-1)/(n-p-1)

adj_r2(r2,n,p)

-0.053884628389635036

### 5.1.2 Dataset Essencials

In [87]:
#Essencials dataset
train_b = train_upd_b.iloc[:,:-1]
target_b = train_upd_b.iloc[:,-1]

In [88]:
# dividing Essencial dataset into train and validation. 
X_train, X_val, y_train, y_val = train_test_split(train_b,target_b, test_size=0.2, random_state=0, stratify=target_b) 

In [89]:
log_model = LogisticRegression()

In [90]:
log_model.fit(X_train,y_train)

LogisticRegression()

In [91]:
y_pred = log_model.predict(X_val)
#y_pred

In [92]:
pred_prob = log_model.predict_proba(X_val)
#pred_prob

In [93]:
log_model.coef_

array([[ 0.16010102, -0.09877765, -0.21262079,  0.28412429,  0.00919461,
         0.36697785,  0.0235715 ,  0.02907623,  0.02918999]])

In [94]:
cm = confusion_matrix(y_val, y_pred)
cm

array([[ 72,  70],
       [ 29, 196]], dtype=int64)

In [95]:
accuracy = accuracy_score(y_val, y_pred)
accuracy

0.7302452316076294

In [96]:
precision = precision_score(y_val, y_pred)
precision

0.7368421052631579

In [97]:
recall_score(y_val, y_pred)

0.8711111111111111

In [98]:
f1 = f1_score(y_val, y_pred)
f1

0.7983706720977597

In [99]:
print('Results on the validation set:')
print(
    # write your code in here
    classification_report(y_true = y_val, y_pred = y_pred)
)

Results on the validation set:
              precision    recall  f1-score   support

           0       0.71      0.51      0.59       142
           1       0.74      0.87      0.80       225

    accuracy                           0.73       367
   macro avg       0.72      0.69      0.70       367
weighted avg       0.73      0.73      0.72       367



### Performance measures

In [100]:
# R2

r2_score(y_val, y_pred)

-0.13718309859154942

In [101]:
# Adjusted R2

r2 = r2_score(y_val, y_pred)
n = len(y_val)
p = len(X_train.columns)

def adj_r2 (r2,n,p):
    return 1-(1-r2)*(n-1)/(n-p-1)

adj_r2(r2,n,p)

-0.1658515800686473

### 5.1.2 Dataset Ticket Buyers

In [102]:
#Ticket buyers
train_c = train_upd_c.iloc[:,:-1]
target_c = train_upd_c.iloc[:,-1]

In [103]:
# dividing Essencial dataset into train and validation. 
X_train, X_val, y_train, y_val = train_test_split(train_c,target_c, test_size=0.2, random_state=0, stratify=target_c) 

In [104]:
log_model = LogisticRegression()

In [105]:
log_model.fit(X_train,y_train)

LogisticRegression()

In [106]:
y_pred = log_model.predict(X_val)
#y_pred

In [107]:
pred_prob = log_model.predict_proba(X_val)
#pred_prob

In [108]:
log_model.coef_

array([[ 0.0730893 , -0.07116122, -0.32400253,  0.50358939,  0.34815191,
         0.09876325,  0.02568445]])

In [109]:
cm = confusion_matrix(y_val, y_pred)
cm

array([[70, 34],
       [25, 83]], dtype=int64)

In [110]:
accuracy = accuracy_score(y_val, y_pred)
accuracy

0.7216981132075472

In [111]:
precision = precision_score(y_val, y_pred)
precision

0.7094017094017094

In [112]:
recall_score(y_val, y_pred)

0.7685185185185185

In [113]:
f1 = f1_score(y_val, y_pred)
f1

0.7377777777777778

In [114]:
print('Results on the validation set:')
print(
    # write your code in here
    classification_report(y_true = y_val, y_pred = y_pred)
)

Results on the validation set:
              precision    recall  f1-score   support

           0       0.74      0.67      0.70       104
           1       0.71      0.77      0.74       108

    accuracy                           0.72       212
   macro avg       0.72      0.72      0.72       212
weighted avg       0.72      0.72      0.72       212



# 4.3 Testing Data

# 5) Applying ML techniques

# 6) Training

# 7) Evaluation / Assessment

# 8) Testing across the ML algos tested above

# 9) Adding to the analysis other ML algos

# 10) Optimization

# 11) Conclussion